In [27]:
import pandas as pd, numpy as np
import os, time, gc
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
pd.options.mode.chained_assignment = None

In [2]:
os.listdir('input')

['.ipynb_checkpoints',
 'new_te.feather',
 'new_te_abl.feather',
 'new_te_cam.feather',
 'new_te_cam_ts.feather',
 'new_te_eve_ts.feather',
 'new_te_rc.feather',
 'new_te_rc_ts.feather',
 'new_tr.feather',
 'new_tr_abl.feather',
 'new_tr_cam.feather',
 'new_tr_cam_ts.feather',
 'new_tr_eve_ts.feather',
 'new_tr_rc.feather',
 'new_tr_rc_ts.feather',
 'sample_submission.csv',
 'te.feather',
 'test.csv',
 'te_abl.feather',
 'te_cam.feather',
 'te_rc.feather',
 'te_v7.feather',
 'tr.feather',
 'train.csv',
 'tr_abl.feather',
 'tr_rc.feather',
 'tr_v7.feather']

In [3]:
%%time
tr = pd.read_feather('../dacon_sc/input/tr.feather')
te = pd.read_feather('../dacon_sc/input/te.feather')
submission = pd.read_csv('../dacon_sc/input/sample_submission.csv')

Wall time: 41.6 s


In [4]:
tr.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


In [5]:
def make_timespan(df):

    df['time_span'] = 0
    df['time_span'] = np.where(df['time']<1, 0, 
                                  np.where(df['time']<2, 1, 
                                          np.where(df['time']<3, 2,
                                                  np.where(df['time']<4, 3,
                                                          np.where(df['time']<5, 4,
                                                                  np.where(df['time']<6, 5,
                                                                          np.where(df['time']<7, 6,
                                                                                  np.where(df['time']<8, 7,
                                                                                          np.where(df['time']<9, 8,
                                                                                                   np.where(df['time']<10, 9, 10))))))))))
    return df

In [6]:
tr.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


# Event

- time span 기준으로 event activity count
> ability 세부적으로 건드리기?

In [7]:
tr['game_id'].unique()

array([    0,     1,     2, ..., 38869, 38870, 38871], dtype=int64)

In [8]:
tr['event'].unique().tolist()

['Camera',
 'Selection',
 'Ability',
 'Right Click',
 'SetControlGroup',
 'GetControlGroup',
 'AddToControlGroup',
 'ControlGroup']

In [9]:
events = tr['event'].unique().tolist()

def event_timespan(df):
    # make timespan
    df = make_timespan(df)
    new_p0 = pd.DataFrame({"game_id":df['game_id'].unique()})
    new_p1 = pd.DataFrame({"game_id":df['game_id'].unique()})
    # for player 0
    for eve_name in tqdm(events, total=len(events)):
        p0 = df[df['player']==0]
        tmp = p0[df['event']==eve_name].groupby(['game_id','event'])['time_span'].value_counts(dropna=False, normalize=False).unstack()
        tmp.columns = ['p0_{}_tsp_'.format(eve_name)+str(i) for i in range(0,11)]
        tmp.reset_index(inplace=True)
        tmp.fillna(0, inplace=True)
        tmp.drop(['event'],axis=1,inplace=True)
        
        new_p0 = new_p0.merge(tmp, on=['game_id'],how='left')
        del tmp;gc.collect()
    del p0;gc.collect()
    
    # for player 0
    for eve_name in tqdm(events, total=len(events)):
        p1 = df[df['player']==1]
        tmp = p1[df['event']==eve_name].groupby(['game_id','event'])['time_span'].value_counts(dropna=False, normalize=False).unstack()
        tmp.columns = ['p1_{}_tsp_'.format(eve_name)+str(i) for i in range(0,11)]
        tmp.reset_index(inplace=True)
        tmp.fillna(0, inplace=True)
        tmp.drop(['event'],axis=1,inplace=True)
        
        new_p1 = new_p1.merge(tmp, on=['game_id'],how='left')
    del p1;gc.collect()
    
    new_df = new_p0.merge(new_p1, on=['game_id'], how='left')
    
    return new_df

In [ ]:
%%time
tr_eve = event_timespan(tr)
te_eve = event_timespan(te)

In [12]:
print(tr_eve.shape, te_eve.shape)

(38872, 177) (16787, 177)


In [13]:
tr_eve.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Camera_tsp_6,p0_Camera_tsp_7,p0_Camera_tsp_8,...,p1_ControlGroup_tsp_1,p1_ControlGroup_tsp_2,p1_ControlGroup_tsp_3,p1_ControlGroup_tsp_4,p1_ControlGroup_tsp_5,p1_ControlGroup_tsp_6,p1_ControlGroup_tsp_7,p1_ControlGroup_tsp_8,p1_ControlGroup_tsp_9,p1_ControlGroup_tsp_10
0,0,45.0,37.0,70.0,76.0,61.0,60.0,64.0,31.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,20.0,39.0,90.0,41.0,59.0,76.0,61.0,75.0,89.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,48.0,23.0,20.0,44.0,50.0,39.0,40.0,30.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,3,27.0,43.0,71.0,63.0,69.0,104.0,65.0,88.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,30.0,69.0,102.0,82.0,72.0,75.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
tr_eve.filter(regex='p0').columns

Index(['p0_Camera_tsp_0', 'p0_Camera_tsp_1', 'p0_Camera_tsp_2',
       'p0_Camera_tsp_3', 'p0_Camera_tsp_4', 'p0_Camera_tsp_5',
       'p0_Camera_tsp_6', 'p0_Camera_tsp_7', 'p0_Camera_tsp_8',
       'p0_Camera_tsp_9', 'p0_Camera_tsp_10', 'p0_Selection_tsp_0',
       'p0_Selection_tsp_1', 'p0_Selection_tsp_2', 'p0_Selection_tsp_3',
       'p0_Selection_tsp_4', 'p0_Selection_tsp_5', 'p0_Selection_tsp_6',
       'p0_Selection_tsp_7', 'p0_Selection_tsp_8', 'p0_Selection_tsp_9',
       'p0_Selection_tsp_10', 'p0_Ability_tsp_0', 'p0_Ability_tsp_1',
       'p0_Ability_tsp_2', 'p0_Ability_tsp_3', 'p0_Ability_tsp_4',
       'p0_Ability_tsp_5', 'p0_Ability_tsp_6', 'p0_Ability_tsp_7',
       'p0_Ability_tsp_8', 'p0_Ability_tsp_9', 'p0_Ability_tsp_10',
       'p0_Right Click_tsp_0', 'p0_Right Click_tsp_1', 'p0_Right Click_tsp_2',
       'p0_Right Click_tsp_3', 'p0_Right Click_tsp_4', 'p0_Right Click_tsp_5',
       'p0_Right Click_tsp_6', 'p0_Right Click_tsp_7', 'p0_Right Click_tsp_8',
       'p0_R

In [17]:
tr_eve.filter(regex='p1').columns

Index(['p1_Camera_tsp_0', 'p1_Camera_tsp_1', 'p1_Camera_tsp_2',
       'p1_Camera_tsp_3', 'p1_Camera_tsp_4', 'p1_Camera_tsp_5',
       'p1_Camera_tsp_6', 'p1_Camera_tsp_7', 'p1_Camera_tsp_8',
       'p1_Camera_tsp_9', 'p1_Camera_tsp_10', 'p1_Selection_tsp_0',
       'p1_Selection_tsp_1', 'p1_Selection_tsp_2', 'p1_Selection_tsp_3',
       'p1_Selection_tsp_4', 'p1_Selection_tsp_5', 'p1_Selection_tsp_6',
       'p1_Selection_tsp_7', 'p1_Selection_tsp_8', 'p1_Selection_tsp_9',
       'p1_Selection_tsp_10', 'p1_Ability_tsp_0', 'p1_Ability_tsp_1',
       'p1_Ability_tsp_2', 'p1_Ability_tsp_3', 'p1_Ability_tsp_4',
       'p1_Ability_tsp_5', 'p1_Ability_tsp_6', 'p1_Ability_tsp_7',
       'p1_Ability_tsp_8', 'p1_Ability_tsp_9', 'p1_Ability_tsp_10',
       'p1_Right Click_tsp_0', 'p1_Right Click_tsp_1', 'p1_Right Click_tsp_2',
       'p1_Right Click_tsp_3', 'p1_Right Click_tsp_4', 'p1_Right Click_tsp_5',
       'p1_Right Click_tsp_6', 'p1_Right Click_tsp_7', 'p1_Right Click_tsp_8',
       'p1_R

In [19]:
p0_cols = tr_eve[tr_eve.filter(regex='p0').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()
p1_cols = tr_eve[tr_eve.filter(regex='p1').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()

print(len(p0_cols),len(p1_cols))

88 88


In [20]:
tr_eve.fillna(0 , inplace=True)
te_eve.fillna(0 , inplace=True)

In [21]:
tr_eve.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Camera_tsp_6,p0_Camera_tsp_7,p0_Camera_tsp_8,...,p1_ControlGroup_tsp_1,p1_ControlGroup_tsp_2,p1_ControlGroup_tsp_3,p1_ControlGroup_tsp_4,p1_ControlGroup_tsp_5,p1_ControlGroup_tsp_6,p1_ControlGroup_tsp_7,p1_ControlGroup_tsp_8,p1_ControlGroup_tsp_9,p1_ControlGroup_tsp_10
0,0,45.0,37.0,70.0,76.0,61.0,60.0,64.0,31.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,20.0,39.0,90.0,41.0,59.0,76.0,61.0,75.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,48.0,23.0,20.0,44.0,50.0,39.0,40.0,30.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,3,27.0,43.0,71.0,63.0,69.0,104.0,65.0,88.0,63.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,30.0,69.0,102.0,82.0,72.0,75.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
def diff_all(df):
    p0_cols = df[df.filter(regex='p0').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()
    p1_cols = df[df.filter(regex='p1').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()
    
    for a, b in tqdm(zip(p0_cols, p1_cols), total=len(p0_cols)):
        df["{}_{}_diff".format(a,b)] = df[a] - df[b]
    
    return df

In [23]:
%%time
tr_eve = diff_all(tr_eve)
te_eve = diff_all(te_eve)

100%|████████████████████████████████████████████████████████████████████████████████| 88/88 [00:00<00:00, 2192.49it/s]

Wall time: 210 ms


In [24]:
tr_eve.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Camera_tsp_6,p0_Camera_tsp_7,p0_Camera_tsp_8,...,p0_ControlGroup_tsp_1_p1_ControlGroup_tsp_1_diff,p0_ControlGroup_tsp_2_p1_ControlGroup_tsp_2_diff,p0_ControlGroup_tsp_3_p1_ControlGroup_tsp_3_diff,p0_ControlGroup_tsp_4_p1_ControlGroup_tsp_4_diff,p0_ControlGroup_tsp_5_p1_ControlGroup_tsp_5_diff,p0_ControlGroup_tsp_6_p1_ControlGroup_tsp_6_diff,p0_ControlGroup_tsp_7_p1_ControlGroup_tsp_7_diff,p0_ControlGroup_tsp_8_p1_ControlGroup_tsp_8_diff,p0_ControlGroup_tsp_9_p1_ControlGroup_tsp_9_diff,p0_ControlGroup_tsp_10_p1_ControlGroup_tsp_10_diff
0,0,45.0,37.0,70.0,76.0,61.0,60.0,64.0,31.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,20.0,39.0,90.0,41.0,59.0,76.0,61.0,75.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,48.0,23.0,20.0,44.0,50.0,39.0,40.0,30.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0
3,3,27.0,43.0,71.0,63.0,69.0,104.0,65.0,88.0,63.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,30.0,69.0,102.0,82.0,72.0,75.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
def cam_feat(df):

    cam = df[df['event']=="Camera"]
    
    cam['cam_lat'] = cam['event_contents'].apply(lambda x: x.split("(")[1].split(",")[0])
    cam['cam_lon'] = cam['event_contents'].apply(lambda x: x.split("(")[1].split(",")[1].split(" ")[1].split(")")[0])

    cam['cam_lat'] = cam['cam_lat'].astype(float)
    cam['cam_lon'] = cam['cam_lon'].astype(float)
    
    cam['cam_lat_lon'] = np.round(cam['cam_lat'], 2).astype(str) + np.round(cam['cam_lon'], 2).astype(str)
    cam['cam_lat_lon2'] = np.round(cam['cam_lat'], 2).astype(str) + np.round(cam['cam_lon'], 2).astype(str)
    
    return cam 

In [26]:
%%time
tr_cam = cam_feat(tr)
te_cam = cam_feat(te)

Wall time: 2min 47s


In [28]:
tr_cam.head()

,game_id,winner,time,player,species,event,event_contents,time_span,cam_lat,cam_lon,cam_lat_lon,cam_lat_lon2
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)",0,145.250000,21.507812,145.2521.51,145.2521.51
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)",0,22.750000,147.007812,22.75147.01,22.75147.01
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)",0,142.996094,24.503906,143.024.5,143.024.5
5,0,1,0.14,0,T,Camera,"at (142.5078125, 24.98828125)",0,142.507812,24.988281,142.5124.99,142.5124.99
6,0,1,0.14,0,T,Camera,"at (139.6171875, 27.8828125)",0,139.617188,27.882812,139.6227.88,139.6227.88


### Start point lat lon and LE

In [30]:
def get_sp(df):

    df_tmp = pd.DataFrame(df.game_id.unique(), columns=['game_id'])
    df_tmp.index = df_tmp.game_id
    df_tmp = df_tmp.drop(['game_id'], axis = 1)
    
    p0 = df[(df.event=='Camera')&(df.player==0)]
    p0 = p0[p0.shift(1).game_id!=p0.game_id] # 쉬프트를 이용하여 각 게임의 첫번째 데이터 찾기
    p0 = p0.loc[:, ['game_id','event_contents']].rename({'event_contents':'player0_starting'}, axis = 1)
    p0.index = p0['game_id']
    p0 = p0.drop(['game_id'], axis=1)
    df_tmp = pd.merge(df_tmp, p0, on='game_id', how='left')
    del p0
    p1 = df[(df.event=='Camera')&(df.player==1)]
    p1 = p1[p1.shift(1).game_id!=p1.game_id]
    p1 = p1.loc[:, ['game_id','event_contents']].rename({'event_contents':'player1_starting'}, axis = 1)
    p1.index = p1['game_id']
    p1 = p1.drop(['game_id'], axis=1)
    df_tmp = pd.merge(df_tmp, p1, on='game_id', how='left')
    del p1
    
    df_tmp['player0_starting'] = df_tmp.player0_starting.str.split('(').str[1]
    df_tmp['player0_starting'] = df_tmp.player0_starting.str.split(')').str[0]
    split_xy = df_tmp.player0_starting.str.split(',')
    df_tmp['player0_x'] = split_xy.str[0].astype('float')
    df_tmp['player0_y'] = split_xy.str[1].astype('float')
    del split_xy

    df_tmp['player1_starting'] = df_tmp.player1_starting.str.split('(').str[1]
    df_tmp['player1_starting'] = df_tmp.player1_starting.str.split(')').str[0]
    split_xy = df_tmp.player1_starting.str.split(',')
    df_tmp['player1_x'] = split_xy.str[0].astype('float')
    df_tmp['player1_y'] = split_xy.str[1].astype('float')
    del split_xy
    
#     location_p0 = df_tmp.loc[:, ['player0_x', 'player0_y']]
#     location_p0 = location_p0.rename({'player0_x':'location_x', 'player0_y':'location_y'}, axis=1)

#     location_p1 = df_tmp.loc[:, ['player1_x', 'player1_y']]
#     location_p1 = location_p1.rename({'player1_x':'location_x', 'player1_y':'location_y'}, axis=1)
#     location_p1.index += location_p0.index[-1]+1
    
#     location = pd.concat([location_p0, location_p1])
#     location = location.dropna()
#     del location_p0, location_p1, df_tmp
    
    return df_tmp

In [64]:
%%time
tr_sp = get_sp(tr)
te_sp = get_sp(te)

Wall time: 27.9 s


In [65]:
tr_sp.head()

,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y
game_id,,,,,,
0,"145.25, 21.5078125","22.75, 147.0078125",145.25,21.507812,22.75,147.007812
1,"140.0, 162.0078125","28.0, 18.5078125",140.00,162.007812,28.00,18.507812
2,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812
3,"127.25, 27.5078125","24.75, 137.0078125",127.25,27.507812,24.75,137.007812
4,"36.75, 122.0078125","163.25, 18.5078125",36.75,122.007812,163.25,18.507812


In [37]:
def df_copy(tr_df, te_df):
    tr = tr_df.copy();te = te_df.copy()
    return tr, te

def lat_lon_create(tr_df, te_df):
    tr, te = df_copy(tr_df, te_df)
    tr['p0_lat_lon'] = np.round(tr['player0_x'], 2).astype(str) + np.round(tr['player0_y'], 2).astype(str)
    tr['p1_lat_lon'] = np.round(tr['player1_x'], 2).astype(str) + np.round(tr['player1_y'], 2).astype(str)
    tr['game_lat_lon'] = tr['p0_lat_lon'] + tr['p1_lat_lon']
    
    te['p0_lat_lon'] = np.round(te['player0_x'], 2).astype(str) + np.round(te['player0_y'], 2).astype(str)
    te['p1_lat_lon'] = np.round(te['player1_x'], 2).astype(str) + np.round(te['player1_y'], 2).astype(str)
    te['game_lat_lon'] = te['p0_lat_lon'] + te['p1_lat_lon']
    
    le = LabelEncoder().fit(pd.concat([tr['p0_lat_lon'], te['p0_lat_lon']]))
    tr['p0_lat_lon_le'] = le.transform(tr['p0_lat_lon'])
    te['p0_lat_lon_le'] = le.transform(te['p0_lat_lon'])
    
    le = LabelEncoder().fit(pd.concat([tr['p1_lat_lon'], te['p1_lat_lon']]))
    tr['p1_lat_lon_le'] = le.transform(tr['p1_lat_lon'])
    te['p1_lat_lon_le'] = le.transform(te['p1_lat_lon'])

    le = LabelEncoder().fit(pd.concat([tr['game_lat_lon'], te['game_lat_lon']]))
    tr['game_lat_lon_le'] = le.transform(tr['game_lat_lon'])
    te['game_lat_lon_le'] = le.transform(te['game_lat_lon'])
    return tr, te

In [66]:
%%time
tr_sp, te_sp = lat_lon_create(tr_sp, te_sp)

Wall time: 130 ms


In [67]:
tr_sp.head()

,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y,p0_lat_lon,p1_lat_lon,game_lat_lon,p0_lat_lon_le,p1_lat_lon_le,game_lat_lon_le
game_id,,,,,,,,,,,,
0,"145.25, 21.5078125","22.75, 147.0078125",145.25,21.507812,22.75,147.007812,145.2521.51,22.75147.01,145.2521.5122.75147.01,314,753,613
1,"140.0, 162.0078125","28.0, 18.5078125",140.00,162.007812,28.00,18.507812,140.0162.01,28.018.51,140.0162.0128.018.51,254,1010,476
2,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812,151.2520.51,24.75128.01,151.2520.5124.75128.01,436,876,853
3,"127.25, 27.5078125","24.75, 137.0078125",127.25,27.507812,24.75,137.007812,127.2527.51,24.75137.01,127.2527.5124.75137.01,113,924,215
4,"36.75, 122.0078125","163.25, 18.5078125",36.75,122.007812,163.25,18.507812,36.75122.01,163.2518.51,36.75122.01163.2518.51,1140,632,2375


In [68]:
te_sp.head()

,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y,p0_lat_lon,p1_lat_lon,game_lat_lon,p0_lat_lon_le,p1_lat_lon_le,game_lat_lon_le
game_id,,,,,,,,,,,,
38872,"22.25, 81.5078125","120.25, 153.83984375",22.25,81.507812,120.25,153.839844,22.2581.51,120.25153.84,22.2581.51120.25153.84,693,32,1354
38873,"24.75, 128.0078125","151.25, 20.5078125",24.75,128.007812,151.25,20.507812,24.75128.01,151.2520.51,24.75128.01151.2520.51,837,457,1688
38874,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812,151.2520.51,24.75128.01,151.2520.5124.75128.01,436,876,853
38875,"152.25, 142.0078125","31.75, 22.5078125",152.25,142.007812,31.75,22.507812,152.25142.01,31.7522.51,152.25142.0131.7522.51,528,1069,1028
38876,"135.25, 31.5078125","22.25, 81.5078125",135.25,31.507812,22.25,81.507812,135.2531.51,22.2581.51,135.2531.5122.2581.51,203,706,392


In [69]:
tr_sp['game_lat_lon_le'].max()

2536

In [70]:
te_sp['game_lat_lon_le'].max()

2537

In [71]:
def spc_df(df):
    p0 = df[df['player']==0].groupby(['game_id'])['species'].max().reset_index().rename(columns={"species":"p0_spc"})
    p1 = df[df['player']==1].groupby(['game_id'])['species'].max().reset_index().rename(columns={"species":"p1_spc"})
    new_df = p0.merge(p1, on='game_id',how='left')
    new_df['game_spc'] = new_df['p0_spc']+new_df['p1_spc']
    return new_df

In [72]:
%%time
tr_spc = spc_df(tr)
te_spc = spc_df(te)

Wall time: 24.9 s


In [73]:
tr_spc.head()

,game_id,p0_spc,p1_spc,game_spc
0,0,T,T,TT
1,1,P,T,PT
2,2,P,Z,PZ
3,3,T,P,TP
4,4,T,Z,TZ


In [74]:
tr_spc.nunique()

game_id     38872
p0_spc          3
p1_spc          3
game_spc        9
dtype: int64

In [75]:
tr_sp = tr_sp.merge(tr_spc, on='game_id', how='left')
te_sp = te_sp.merge(te_spc, on='game_id', how='left')
tr_sp.head()

,game_id,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y,p0_lat_lon,p1_lat_lon,game_lat_lon,p0_lat_lon_le,p1_lat_lon_le,game_lat_lon_le,p0_spc,p1_spc,game_spc
0,0,"145.25, 21.5078125","22.75, 147.0078125",145.25,21.507812,22.75,147.007812,145.2521.51,22.75147.01,145.2521.5122.75147.01,314,753,613,T,T,TT
1,1,"140.0, 162.0078125","28.0, 18.5078125",140.00,162.007812,28.00,18.507812,140.0162.01,28.018.51,140.0162.0128.018.51,254,1010,476,P,T,PT
2,2,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812,151.2520.51,24.75128.01,151.2520.5124.75128.01,436,876,853,P,Z,PZ
3,3,"127.25, 27.5078125","24.75, 137.0078125",127.25,27.507812,24.75,137.007812,127.2527.51,24.75137.01,127.2527.5124.75137.01,113,924,215,T,P,TP
4,4,"36.75, 122.0078125","163.25, 18.5078125",36.75,122.007812,163.25,18.507812,36.75122.01,163.2518.51,36.75122.01163.2518.51,1140,632,2375,T,Z,TZ


In [76]:
def lat_lon_spc_create(tr_df, te_df):
    tr, te = df_copy(tr_df, te_df)
    tr['p0_latlon_spc'] = tr['p0_lat_lon'] + tr['p0_spc']
    tr['p1_latlon_spc'] = tr['p1_lat_lon'] + tr['p1_spc']
    tr['game_latlon_spc'] = tr['p0_latlon_spc'] + tr['p1_latlon_spc']
    
    te['p0_latlon_spc'] = te['p0_lat_lon'] + te['p0_spc']
    te['p1_latlon_spc'] = te['p1_lat_lon'] + te['p1_spc']
    te['game_latlon_spc'] = te['p0_latlon_spc'] + te['p1_latlon_spc']
    
    le = LabelEncoder().fit(pd.concat([tr['p0_latlon_spc'], te['p0_latlon_spc']]))
    tr['p0_latlon_spc_le'] = le.transform(tr['p0_latlon_spc'])
    te['p0_latlon_spc_le'] = le.transform(te['p0_latlon_spc'])
    
    le = LabelEncoder().fit(pd.concat([tr['p1_latlon_spc'], te['p1_latlon_spc']]))
    tr['p1_latlon_spc_le'] = le.transform(tr['p1_latlon_spc'])
    te['p1_latlon_spc_le'] = le.transform(te['p1_latlon_spc'])
    
    le = LabelEncoder().fit(pd.concat([tr['game_latlon_spc'], te['game_latlon_spc']]))
    tr['game_latlon_spc_le'] = le.transform(tr['game_latlon_spc'])
    te['game_latlon_spc_le'] = le.transform(te['game_latlon_spc'])    
    
    return tr, te

In [77]:
tr_sp.head()

,game_id,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y,p0_lat_lon,p1_lat_lon,game_lat_lon,p0_lat_lon_le,p1_lat_lon_le,game_lat_lon_le,p0_spc,p1_spc,game_spc
0,0,"145.25, 21.5078125","22.75, 147.0078125",145.25,21.507812,22.75,147.007812,145.2521.51,22.75147.01,145.2521.5122.75147.01,314,753,613,T,T,TT
1,1,"140.0, 162.0078125","28.0, 18.5078125",140.00,162.007812,28.00,18.507812,140.0162.01,28.018.51,140.0162.0128.018.51,254,1010,476,P,T,PT
2,2,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812,151.2520.51,24.75128.01,151.2520.5124.75128.01,436,876,853,P,Z,PZ
3,3,"127.25, 27.5078125","24.75, 137.0078125",127.25,27.507812,24.75,137.007812,127.2527.51,24.75137.01,127.2527.5124.75137.01,113,924,215,T,P,TP
4,4,"36.75, 122.0078125","163.25, 18.5078125",36.75,122.007812,163.25,18.507812,36.75122.01,163.2518.51,36.75122.01163.2518.51,1140,632,2375,T,Z,TZ


In [78]:
te_sp.head()

,game_id,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y,p0_lat_lon,p1_lat_lon,game_lat_lon,p0_lat_lon_le,p1_lat_lon_le,game_lat_lon_le,p0_spc,p1_spc,game_spc
0,38872,"22.25, 81.5078125","120.25, 153.83984375",22.25,81.507812,120.25,153.839844,22.2581.51,120.25153.84,22.2581.51120.25153.84,693,32,1354,P,P,PP
1,38873,"24.75, 128.0078125","151.25, 20.5078125",24.75,128.007812,151.25,20.507812,24.75128.01,151.2520.51,24.75128.01151.2520.51,837,457,1688,Z,P,ZP
2,38874,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812,151.2520.51,24.75128.01,151.2520.5124.75128.01,436,876,853,T,T,TT
3,38875,"152.25, 142.0078125","31.75, 22.5078125",152.25,142.007812,31.75,22.507812,152.25142.01,31.7522.51,152.25142.0131.7522.51,528,1069,1028,T,P,TP
4,38876,"135.25, 31.5078125","22.25, 81.5078125",135.25,31.507812,22.25,81.507812,135.2531.51,22.2581.51,135.2531.5122.2581.51,203,706,392,Z,Z,ZZ


In [79]:
%%time
tr_sp, te_sp = lat_lon_spc_create(tr_sp, te_sp)

Wall time: 59.9 ms


In [80]:
tr_sp.head()

,game_id,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y,p0_lat_lon,p1_lat_lon,game_lat_lon,...,game_lat_lon_le,p0_spc,p1_spc,game_spc,p0_latlon_spc,p1_latlon_spc,game_latlon_spc,p0_latlon_spc_le,p1_latlon_spc_le,game_latlon_spc_le
0,0,"145.25, 21.5078125","22.75, 147.0078125",145.25,21.507812,22.75,147.007812,145.2521.51,22.75147.01,145.2521.5122.75147.01,...,613,T,T,TT,145.2521.51T,22.75147.01T,145.2521.51T22.75147.01T,359,859,883
1,1,"140.0, 162.0078125","28.0, 18.5078125",140.00,162.007812,28.00,18.507812,140.0162.01,28.018.51,140.0162.0128.018.51,...,476,P,T,PT,140.0162.01P,28.018.51T,140.0162.01P28.018.51T,284,1156,650
2,2,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812,151.2520.51,24.75128.01,151.2520.5124.75128.01,...,853,P,Z,PZ,151.2520.51P,24.75128.01Z,151.2520.51P24.75128.01Z,492,995,1148
3,3,"127.25, 27.5078125","24.75, 137.0078125",127.25,27.507812,24.75,137.007812,127.2527.51,24.75137.01,127.2527.5124.75137.01,...,215,T,P,TP,127.2527.51T,24.75137.01P,127.2527.51T24.75137.01P,125,1049,295
4,4,"36.75, 122.0078125","163.25, 18.5078125",36.75,122.007812,163.25,18.507812,36.75122.01,163.2518.51,36.75122.01163.2518.51,...,2375,T,Z,TZ,36.75122.01T,163.2518.51Z,36.75122.01T163.2518.51Z,1308,720,3275


In [81]:
te_sp.head()

,game_id,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y,p0_lat_lon,p1_lat_lon,game_lat_lon,...,game_lat_lon_le,p0_spc,p1_spc,game_spc,p0_latlon_spc,p1_latlon_spc,game_latlon_spc,p0_latlon_spc_le,p1_latlon_spc_le,game_latlon_spc_le
0,38872,"22.25, 81.5078125","120.25, 153.83984375",22.25,81.507812,120.25,153.839844,22.2581.51,120.25153.84,22.2581.51120.25153.84,...,1354,P,P,PP,22.2581.51P,120.25153.84P,22.2581.51P120.25153.84P,784,34,1831
1,38873,"24.75, 128.0078125","151.25, 20.5078125",24.75,128.007812,151.25,20.507812,24.75128.01,151.2520.51,24.75128.01151.2520.51,...,1688,Z,P,ZP,24.75128.01Z,151.2520.51P,24.75128.01Z151.2520.51P,954,520,2358
2,38874,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812,151.2520.51,24.75128.01,151.2520.5124.75128.01,...,853,T,T,TT,151.2520.51T,24.75128.01T,151.2520.51T24.75128.01T,493,994,1188
3,38875,"152.25, 142.0078125","31.75, 22.5078125",152.25,142.007812,31.75,22.507812,152.25142.01,31.7522.51,152.25142.0131.7522.51,...,1028,T,P,TP,152.25142.01T,31.7522.51P,152.25142.01T31.7522.51P,597,1223,1419
4,38876,"135.25, 31.5078125","22.25, 81.5078125",135.25,31.507812,22.25,81.507812,135.2531.51,22.2581.51,135.2531.5122.2581.51,...,392,Z,Z,ZZ,135.2531.51Z,22.2581.51Z,135.2531.51Z22.2581.51Z,227,806,561


In [82]:
tr.head()

,game_id,winner,time,player,species,event,event_contents,time_span
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)",0
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)",0
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]'],0
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV,0
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)",0


In [83]:
tr_eve.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Camera_tsp_6,p0_Camera_tsp_7,p0_Camera_tsp_8,...,p0_ControlGroup_tsp_1_p1_ControlGroup_tsp_1_diff,p0_ControlGroup_tsp_2_p1_ControlGroup_tsp_2_diff,p0_ControlGroup_tsp_3_p1_ControlGroup_tsp_3_diff,p0_ControlGroup_tsp_4_p1_ControlGroup_tsp_4_diff,p0_ControlGroup_tsp_5_p1_ControlGroup_tsp_5_diff,p0_ControlGroup_tsp_6_p1_ControlGroup_tsp_6_diff,p0_ControlGroup_tsp_7_p1_ControlGroup_tsp_7_diff,p0_ControlGroup_tsp_8_p1_ControlGroup_tsp_8_diff,p0_ControlGroup_tsp_9_p1_ControlGroup_tsp_9_diff,p0_ControlGroup_tsp_10_p1_ControlGroup_tsp_10_diff
0,0,45.0,37.0,70.0,76.0,61.0,60.0,64.0,31.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,20.0,39.0,90.0,41.0,59.0,76.0,61.0,75.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,48.0,23.0,20.0,44.0,50.0,39.0,40.0,30.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0
3,3,27.0,43.0,71.0,63.0,69.0,104.0,65.0,88.0,63.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,30.0,69.0,102.0,82.0,72.0,75.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
tr_sp.head()

,game_id,player0_starting,player1_starting,player0_x,player0_y,player1_x,player1_y,p0_lat_lon,p1_lat_lon,game_lat_lon,...,game_lat_lon_le,p0_spc,p1_spc,game_spc,p0_latlon_spc,p1_latlon_spc,game_latlon_spc,p0_latlon_spc_le,p1_latlon_spc_le,game_latlon_spc_le
0,0,"145.25, 21.5078125","22.75, 147.0078125",145.25,21.507812,22.75,147.007812,145.2521.51,22.75147.01,145.2521.5122.75147.01,...,613,T,T,TT,145.2521.51T,22.75147.01T,145.2521.51T22.75147.01T,359,859,883
1,1,"140.0, 162.0078125","28.0, 18.5078125",140.00,162.007812,28.00,18.507812,140.0162.01,28.018.51,140.0162.0128.018.51,...,476,P,T,PT,140.0162.01P,28.018.51T,140.0162.01P28.018.51T,284,1156,650
2,2,"151.25, 20.5078125","24.75, 128.0078125",151.25,20.507812,24.75,128.007812,151.2520.51,24.75128.01,151.2520.5124.75128.01,...,853,P,Z,PZ,151.2520.51P,24.75128.01Z,151.2520.51P24.75128.01Z,492,995,1148
3,3,"127.25, 27.5078125","24.75, 137.0078125",127.25,27.507812,24.75,137.007812,127.2527.51,24.75137.01,127.2527.5124.75137.01,...,215,T,P,TP,127.2527.51T,24.75137.01P,127.2527.51T24.75137.01P,125,1049,295
4,4,"36.75, 122.0078125","163.25, 18.5078125",36.75,122.007812,163.25,18.507812,36.75122.01,163.2518.51,36.75122.01163.2518.51,...,2375,T,Z,TZ,36.75122.01T,163.2518.51Z,36.75122.01T163.2518.51Z,1308,720,3275


In [86]:
new_tr = tr_eve.merge(tr_sp, on='game_id', how='left')
new_te = te_eve.merge(te_sp, on='game_id', how='left')

In [87]:
new_tr.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Camera_tsp_6,p0_Camera_tsp_7,p0_Camera_tsp_8,...,game_lat_lon_le,p0_spc,p1_spc,game_spc,p0_latlon_spc,p1_latlon_spc,game_latlon_spc,p0_latlon_spc_le,p1_latlon_spc_le,game_latlon_spc_le
0,0,45.0,37.0,70.0,76.0,61.0,60.0,64.0,31.0,0.0,...,613,T,T,TT,145.2521.51T,22.75147.01T,145.2521.51T22.75147.01T,359,859,883
1,1,20.0,39.0,90.0,41.0,59.0,76.0,61.0,75.0,89.0,...,476,P,T,PT,140.0162.01P,28.018.51T,140.0162.01P28.018.51T,284,1156,650
2,2,48.0,23.0,20.0,44.0,50.0,39.0,40.0,30.0,89.0,...,853,P,Z,PZ,151.2520.51P,24.75128.01Z,151.2520.51P24.75128.01Z,492,995,1148
3,3,27.0,43.0,71.0,63.0,69.0,104.0,65.0,88.0,63.0,...,215,T,P,TP,127.2527.51T,24.75137.01P,127.2527.51T24.75137.01P,125,1049,295
4,4,30.0,69.0,102.0,82.0,72.0,75.0,0.0,0.0,0.0,...,2375,T,Z,TZ,36.75122.01T,163.2518.51Z,36.75122.01T163.2518.51Z,1308,720,3275


In [241]:
new_tr.to_feather('../dacon_sc/input/tr_tsact_sp.feather')

In [242]:
new_te.to_feather('../dacon_sc/input/te_tsact_sp.feather')

In [89]:
new_tr.select_dtypes(include=[object]).columns.tolist()

['player0_starting',
 'player1_starting',
 'p0_lat_lon',
 'p1_lat_lon',
 'game_lat_lon',
 'p0_spc',
 'p1_spc',
 'game_spc',
 'p0_latlon_spc',
 'p1_latlon_spc',
 'game_latlon_spc']

In [90]:
def category_transform(tr_df, te_df):
    tr, te = df_copy(tr_df, te_df)
    cate_col = tr.select_dtypes(include=[object]).columns.tolist()
    for df in [tr, te]:
        for col in cate_col:
            df[col] = df[col].astype('category')
    return tr, te

In [91]:
new_tr, new_te = category_transform(new_tr, new_te)

In [93]:
new_tr.dtypes.value_counts()

float64     268
int32         6
category      2
category      1
int64         1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
dtype: int64

In [94]:
tr_y = tr.groupby(['game_id'])['winner'].max()
tr_y = tr_y.reset_index()
train_Y = tr_y['winner']
tr_y.head()

,game_id,winner
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0


In [95]:
import time
from sklearn import metrics
from operator import itemgetter
import lightgbm as lgb

In [96]:
from sklearn.model_selection import KFold

def LGB_KFOLD_BINA(n_fold, train_X, test_X, metric, lr, num_leaves, max_depth, min_data_in_leaf):
    
    folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

    oof_lgb = np.zeros(len(train_X))
    predictions = np.zeros(len(test_X))
    feature_importance_df = pd.DataFrame()
    cv_score_df = []

    # Model parameters
    lgb_params = {'num_leaves': num_leaves,
                 'min_data_in_leaf': min_data_in_leaf, 
                 'objective':'binary',
                 'max_depth': max_depth,
                 'learning_rate': lr,
                 "boosting": "gbdt",
                 "feature_fraction": 0.3,
                 "bagging_freq": 1,
                 "bagging_fraction": 0.7,
                 "bagging_seed": 42,
                 "metric": metric,
                 "lambda_l1": 0.0,
                 "verbosity": 300,
                 "nthread": -1,
                 "random_state": 42}

    model_start = time.time()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X)):
        strLog = "fold {}".format(fold_)
        print(strLog+ "-" * 50)

        X_tr, X_val = train_X.iloc[trn_idx], train_X.iloc[val_idx]
        y_tr, y_val = train_Y.iloc[trn_idx], train_Y.iloc[val_idx]

        model = lgb.LGBMClassifier(**lgb_params, n_estimators = 200000, n_jobs = -1)
        model.fit(X_tr, 
                  y_tr, 
                  eval_set=[(X_tr, y_tr), (X_val, y_val)], 
                  eval_metric=metric,
                  verbose=300, 
                  early_stopping_rounds=200)
        oof_lgb[val_idx] = model.predict_proba(X_val, num_iteration=model.best_iteration_)[:,1]

        cv_score_df.append(model.best_score_)

        #feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df["Feature"] = train_X.columns
        fold_importance_df["importance"] = model.feature_importances_[:len(train_X.columns)]
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        #predictions
        predictions += model.predict_proba(test_X, num_iteration=model.best_iteration_)[:,1] / folds.n_splits

    cv_score_df = pd.DataFrame.from_dict(cv_score_df)
    cv_score_df = cv_score_df.valid_1.tolist()
    cv_score_df = list(map(itemgetter(metric),cv_score_df))

    print("-" * 50)
    #print("SF   RMSE = {}".format(oof_score))
    print("Mean "+ metric + " = {}".format(np.mean(cv_score_df)))
    print("Std "+ metric + " = {}".format(np.std(cv_score_df)))
#     lgb.plot_metric(model, metric=metric, title='auc plot', xlabel='Iterations', ylabel='auto', figsize=(10,8), grid=False)

    model_end = time.time()
    model_elapsed = model_end - model_start
    print('Model elapsed {0:0.2f}'.format(model_elapsed/60), "minutes.")
    cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:].index)
    best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]
    pd.set_option('display.max_rows', 500)
    best_features = best_features.drop(['fold'],axis=1)
    best_features = best_features.groupby(['Feature'], as_index = False).mean()
    best_features['Feature Rank'] = best_features['importance'].rank(ascending=0)
    best_features = best_features.sort_values('Feature Rank', ascending = True)
    print(best_features.loc[best_features['importance']!=0].head(100))

    return predictions, oof_lgb, np.mean(cv_score_df), np.std(cv_score_df), best_features

In [97]:
new_tr.dtypes.value_counts()

float64     268
int32         6
category      2
category      1
int64         1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
dtype: int64

In [98]:
#early stopping 200
pred, oof, cv, cv_std, best_feat = LGB_KFOLD_BINA(5, new_tr.drop('game_id',axis=1), new_te.drop('game_id',axis=1), 
                                                  "auc", lr=0.003, num_leaves=700, max_depth=-1, min_data_in_leaf=20)

fold 0--------------------------------------------------
Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.993068	valid_1's auc: 0.640947
[600]	training's auc: 0.99887	valid_1's auc: 0.645935
[900]	training's auc: 0.999858	valid_1's auc: 0.648629
[1200]	training's auc: 0.999984	valid_1's auc: 0.649642
[1500]	training's auc: 0.999998	valid_1's auc: 0.650515
Early stopping, best iteration is:
[1544]	training's auc: 0.999999	valid_1's auc: 0.650695
fold 1--------------------------------------------------
Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.993727	valid_1's auc: 0.659399
[600]	training's auc: 0.998971	valid_1's auc: 0.664087
[900]	training's auc: 0.999872	valid_1's auc: 0.66578
[1200]	training's auc: 0.999986	valid_1's auc: 0.667295
[1500]	training's auc: 0.999998	valid_1's auc: 0.667651
Early stopping, best iteration is:
[1322]	training's auc: 0.999995	valid_1's auc: 0.667902
fold 2----------------------

In [144]:
best_feat

,Feature,importance,Feature Rank
208,p1_Camera_tsp_0,13244.0,1.0
49,p0_Camera_tsp_0,12649.2,2.0
50,p0_Camera_tsp_0_p1_Camera_tsp_0_diff,12592.2,3.0
54,p0_Camera_tsp_1_p1_Camera_tsp_1_diff,12416.2,4.0
56,p0_Camera_tsp_2_p1_Camera_tsp_2_diff,12069.8,5.0
211,p1_Camera_tsp_2,11475.6,6.0
51,p0_Camera_tsp_1,11292.2,7.0
55,p0_Camera_tsp_2,11141.4,8.0
209,p1_Camera_tsp_1,11131.4,9.0
58,p0_Camera_tsp_3_p1_Camera_tsp_3_diff,10993.6,10.0


In [99]:
def day_agg(tr_df, te_df, merge_columns, columns, aggs=['mean']):
    tr, te = df_copy(tr_df, te_df)
    for merge_column in merge_columns:
        for col in columns:
            for agg in aggs:
                valid = pd.concat([tr[[merge_column, col]], te[[merge_column, col]]])
                new_cn = merge_column + '_' + agg + '_' + col
                if agg=='quantile':
                    valid = valid.groupby(merge_column)[col].quantile(0.8).reset_index().rename(columns={col:new_cn})
                else:
                    valid = valid.groupby(merge_column)[col].agg([agg]).reset_index().rename(columns={agg:new_cn})
                valid.index = valid[merge_column].tolist()
                valid = valid[new_cn].to_dict()
            
                tr[new_cn] = tr[merge_column].map(valid)
                te[new_cn] = te[merge_column].map(valid)
    return tr, te

def sub_day_agg(tr_df, te_df, merge_columns, date_columns, columns, aggs=['mean']):
    tr, te = df_copy(tr_df, te_df)
    for merge_column in merge_columns:
        for date in date_columns:
            tr['mc_date'] = tr[merge_column].astype(str) + '_' +tr[date].astype(str)
            te['mc_date'] = te[merge_column].astype(str) + '_' +te[date].astype(str)
            for col in columns:
                for agg in aggs:
                    valid = pd.concat([tr[['mc_date', col]], te[['mc_date', col]]])
                    new_cn = merge_column + '_' + date + '_' + col + '_' + agg
                    if agg=='quantile':
                        valid = valid.groupby('mc_date')[col].quantile(0.8).reset_index().rename(columns={col:new_cn})
                    else:
                        valid = valid.groupby('mc_date')[col].agg([agg]).reset_index().rename(columns={agg:new_cn})
                    valid.index = valid['mc_date'].tolist()
                    valid = valid[new_cn].to_dict()
                
                    tr[new_cn] = tr['mc_date'].map(valid)
                    te[new_cn] = te['mc_date'].map(valid)
    tr = tr.drop(columns=['mc_date'])
    te = te.drop(columns=['mc_date'])
    return tr, te

In [100]:
new_tr.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Camera_tsp_6,p0_Camera_tsp_7,p0_Camera_tsp_8,...,game_lat_lon_le,p0_spc,p1_spc,game_spc,p0_latlon_spc,p1_latlon_spc,game_latlon_spc,p0_latlon_spc_le,p1_latlon_spc_le,game_latlon_spc_le
0,0,45.0,37.0,70.0,76.0,61.0,60.0,64.0,31.0,0.0,...,613,T,T,TT,145.2521.51T,22.75147.01T,145.2521.51T22.75147.01T,359,859,883
1,1,20.0,39.0,90.0,41.0,59.0,76.0,61.0,75.0,89.0,...,476,P,T,PT,140.0162.01P,28.018.51T,140.0162.01P28.018.51T,284,1156,650
2,2,48.0,23.0,20.0,44.0,50.0,39.0,40.0,30.0,89.0,...,853,P,Z,PZ,151.2520.51P,24.75128.01Z,151.2520.51P24.75128.01Z,492,995,1148
3,3,27.0,43.0,71.0,63.0,69.0,104.0,65.0,88.0,63.0,...,215,T,P,TP,127.2527.51T,24.75137.01P,127.2527.51T24.75137.01P,125,1049,295
4,4,30.0,69.0,102.0,82.0,72.0,75.0,0.0,0.0,0.0,...,2375,T,Z,TZ,36.75122.01T,163.2518.51Z,36.75122.01T163.2518.51Z,1308,720,3275


In [147]:
new_tr[['game_latlon_spc','game_latlon_spc_le','p1_latlon_spc_le','p0_latlon_spc_le','player0_starting','player1_y','player0_y','p0_latlon_spc','p1_latlon_spc','player0_x','player1_x','p1_lat_lon_le','p0_lat_lon_le','game_lat_lon_le','p1_spc','p0_spc']]

,game_latlon_spc,game_latlon_spc_le,p1_latlon_spc_le,p0_latlon_spc_le,player0_starting,player1_y,player0_y,p0_latlon_spc,p1_latlon_spc,player0_x,player1_x,p1_lat_lon_le,p0_lat_lon_le,game_lat_lon_le,p1_spc,p0_spc
0,145.2521.51T22.75147.01T,883,859,359,"145.25, 21.5078125",147.007812,21.507812,145.2521.51T,22.75147.01T,145.25,22.75,753,314,613,T,T
1,140.0162.01P28.018.51T,650,1156,284,"140.0, 162.0078125",18.507812,162.007812,140.0162.01P,28.018.51T,140.00,28.00,1010,254,476,T,P
2,151.2520.51P24.75128.01Z,1148,995,492,"151.25, 20.5078125",128.007812,20.507812,151.2520.51P,24.75128.01Z,151.25,24.75,876,436,853,Z,P
3,127.2527.51T24.75137.01P,295,1049,125,"127.25, 27.5078125",137.007812,27.507812,127.2527.51T,24.75137.01P,127.25,24.75,924,113,215,P,T
4,36.75122.01T163.2518.51Z,3275,720,1308,"36.75, 122.0078125",18.507812,122.007812,36.75122.01T,163.2518.51Z,36.75,163.25,632,1140,2375,Z,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38867,22.75147.01T145.2521.51P,2034,366,844,"22.75, 147.0078125",21.507812,147.007812,22.75147.01T,145.2521.51P,22.75,145.25,322,743,1482,P,T
38868,28.018.51T140.0163.75P,2799,296,1114,"28.0, 18.5078125",163.750000,18.507812,28.018.51T,140.0163.75P,28.00,140.00,261,970,2038,P,T
38869,151.2520.51P24.75128.01Z,1148,995,492,"151.25, 20.5078125",128.011719,20.507812,151.2520.51P,24.75128.01Z,151.25,24.75,876,436,853,Z,P
38870,152.25142.01P31.7522.51P,1378,1223,596,"152.25, 142.0078125",22.507812,142.007812,152.25142.01P,31.7522.51P,152.25,31.75,1069,528,1028,P,P


In [ ]:
['game_latlon_spc','game_latlon_spc_le','p1_latlon_spc_le','p0_latlon_spc_le','player1_y','player0_y','p0_latlon_spc','p1_latlon_spc','player0_x','player1_x','p1_lat_lon_le','p0_lat_lon_le','game_lat_lon_le','p1_spc','p0_spc']

In [126]:
new_tr.filter(regex='tsp_').columns[:176]

Index(['p0_Camera_tsp_0', 'p0_Camera_tsp_1', 'p0_Camera_tsp_2',
       'p0_Camera_tsp_3', 'p0_Camera_tsp_4', 'p0_Camera_tsp_5',
       'p0_Camera_tsp_6', 'p0_Camera_tsp_7', 'p0_Camera_tsp_8',
       'p0_Camera_tsp_9',
       ...
       'p1_ControlGroup_tsp_1', 'p1_ControlGroup_tsp_2',
       'p1_ControlGroup_tsp_3', 'p1_ControlGroup_tsp_4',
       'p1_ControlGroup_tsp_5', 'p1_ControlGroup_tsp_6',
       'p1_ControlGroup_tsp_7', 'p1_ControlGroup_tsp_8',
       'p1_ControlGroup_tsp_9', 'p1_ControlGroup_tsp_10'],
      dtype='object', length=176)

In [104]:
new_tr.select_dtypes(include=['category']).columns

Index(['player0_starting', 'player1_starting', 'p0_lat_lon', 'p1_lat_lon',
       'game_lat_lon', 'p0_spc', 'p1_spc', 'game_spc', 'p0_latlon_spc',
       'p1_latlon_spc', 'game_latlon_spc'],
      dtype='object')

In [177]:
# target_cols = best_feat['Feature'][:20]
target_cols = new_tr.filter(regex='tsp_').columns[:176].tolist()
# target_cols = new_tr.filter(regex='tsp_').columns.tolist()
# by_cols = tr_sp.columns[1:].tolist()
by_cols = ['game_latlon_spc']

In [178]:
len(by_cols)

1

In [179]:
new_tr.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Camera_tsp_6,p0_Camera_tsp_7,p0_Camera_tsp_8,...,game_lat_lon_le,p0_spc,p1_spc,game_spc,p0_latlon_spc,p1_latlon_spc,game_latlon_spc,p0_latlon_spc_le,p1_latlon_spc_le,game_latlon_spc_le
0,0,45.0,37.0,70.0,76.0,61.0,60.0,64.0,31.0,0.0,...,613,T,T,TT,145.2521.51T,22.75147.01T,145.2521.51T22.75147.01T,359,859,883
1,1,20.0,39.0,90.0,41.0,59.0,76.0,61.0,75.0,89.0,...,476,P,T,PT,140.0162.01P,28.018.51T,140.0162.01P28.018.51T,284,1156,650
2,2,48.0,23.0,20.0,44.0,50.0,39.0,40.0,30.0,89.0,...,853,P,Z,PZ,151.2520.51P,24.75128.01Z,151.2520.51P24.75128.01Z,492,995,1148
3,3,27.0,43.0,71.0,63.0,69.0,104.0,65.0,88.0,63.0,...,215,T,P,TP,127.2527.51T,24.75137.01P,127.2527.51T24.75137.01P,125,1049,295
4,4,30.0,69.0,102.0,82.0,72.0,75.0,0.0,0.0,0.0,...,2375,T,Z,TZ,36.75122.01T,163.2518.51Z,36.75122.01T163.2518.51Z,1308,720,3275


In [180]:
%%time
day_tr, day_te = day_agg(new_tr, new_te, merge_columns=by_cols, columns=target_cols, aggs=['mean'])

Wall time: 24.7 s


In [181]:
day_tr.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Camera_tsp_6,p0_Camera_tsp_7,p0_Camera_tsp_8,...,game_latlon_spc_mean_p1_ControlGroup_tsp_1,game_latlon_spc_mean_p1_ControlGroup_tsp_2,game_latlon_spc_mean_p1_ControlGroup_tsp_3,game_latlon_spc_mean_p1_ControlGroup_tsp_4,game_latlon_spc_mean_p1_ControlGroup_tsp_5,game_latlon_spc_mean_p1_ControlGroup_tsp_6,game_latlon_spc_mean_p1_ControlGroup_tsp_7,game_latlon_spc_mean_p1_ControlGroup_tsp_8,game_latlon_spc_mean_p1_ControlGroup_tsp_9,game_latlon_spc_mean_p1_ControlGroup_tsp_10
0,0,45.0,37.0,70.0,76.0,61.0,60.0,64.0,31.0,0.0,...,0.037288,0.037288,0.028814,0.028814,0.020339,0.025424,0.023729,0.001695,0.015254,0.001695
1,1,20.0,39.0,90.0,41.0,59.0,76.0,61.0,75.0,89.0,...,0.016667,0.025000,0.016667,0.008333,0.016667,0.027778,0.025000,0.019444,0.000000,0.002778
2,2,48.0,23.0,20.0,44.0,50.0,39.0,40.0,30.0,89.0,...,0.065327,0.103015,0.060302,0.062814,0.035176,0.065327,0.017588,0.067839,0.020101,0.002513
3,3,27.0,43.0,71.0,63.0,69.0,104.0,65.0,88.0,63.0,...,0.050378,0.045340,0.022670,0.052897,0.052897,0.047859,0.025189,0.010076,0.015113,0.000000
4,4,30.0,69.0,102.0,82.0,72.0,75.0,0.0,0.0,0.0,...,0.033708,0.078652,0.040449,0.040449,0.062921,0.069663,0.060674,0.069663,0.056180,0.008989


In [162]:
def frequency_encoding(tr_df, te_df, columns, normalize=False):
    tr, te = df_copy(tr_df, te_df)
    for col in columns:
        if not normalize:
            freq_encode = pd.concat([tr[col], te[col]]).value_counts()
            tr[col+'_fq_enc'] = tr[col].map(freq_encode)
            te[col+'_fq_enc'] = te[col].map(freq_encode)
        else:
            freq_encode = pd.concat([tr[col], te[col]]).value_counts(normalize=True)
            tr[col+'_fq_enc_nor'] = tr[col].map(freq_encode)
            te[col+'_fq_enc_nor'] = te[col].map(freq_encode)
    return tr, te

In [163]:
# %%time
# category_features = new_tr.select_dtypes(include=['category']).columns.tolist()
# day_tr, day_tr = frequency_encoding(day_tr, day_tr, category_features)

In [182]:
day_tr.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Camera_tsp_6,p0_Camera_tsp_7,p0_Camera_tsp_8,...,game_latlon_spc_mean_p1_ControlGroup_tsp_1,game_latlon_spc_mean_p1_ControlGroup_tsp_2,game_latlon_spc_mean_p1_ControlGroup_tsp_3,game_latlon_spc_mean_p1_ControlGroup_tsp_4,game_latlon_spc_mean_p1_ControlGroup_tsp_5,game_latlon_spc_mean_p1_ControlGroup_tsp_6,game_latlon_spc_mean_p1_ControlGroup_tsp_7,game_latlon_spc_mean_p1_ControlGroup_tsp_8,game_latlon_spc_mean_p1_ControlGroup_tsp_9,game_latlon_spc_mean_p1_ControlGroup_tsp_10
0,0,45.0,37.0,70.0,76.0,61.0,60.0,64.0,31.0,0.0,...,0.037288,0.037288,0.028814,0.028814,0.020339,0.025424,0.023729,0.001695,0.015254,0.001695
1,1,20.0,39.0,90.0,41.0,59.0,76.0,61.0,75.0,89.0,...,0.016667,0.025000,0.016667,0.008333,0.016667,0.027778,0.025000,0.019444,0.000000,0.002778
2,2,48.0,23.0,20.0,44.0,50.0,39.0,40.0,30.0,89.0,...,0.065327,0.103015,0.060302,0.062814,0.035176,0.065327,0.017588,0.067839,0.020101,0.002513
3,3,27.0,43.0,71.0,63.0,69.0,104.0,65.0,88.0,63.0,...,0.050378,0.045340,0.022670,0.052897,0.052897,0.047859,0.025189,0.010076,0.015113,0.000000
4,4,30.0,69.0,102.0,82.0,72.0,75.0,0.0,0.0,0.0,...,0.033708,0.078652,0.040449,0.040449,0.062921,0.069663,0.060674,0.069663,0.056180,0.008989


In [183]:
#early stopping 200
pred, oof, cv, cv_std, best_feat = LGB_KFOLD_BINA(5, day_tr.drop('game_id',axis=1), day_te.drop('game_id',axis=1), 
                                                  "auc", lr=0.003, num_leaves=700, max_depth=-1, min_data_in_leaf=20)

fold 0--------------------------------------------------
Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.995229	valid_1's auc: 0.637433
[600]	training's auc: 0.999271	valid_1's auc: 0.640991
[900]	training's auc: 0.999917	valid_1's auc: 0.645027
[1200]	training's auc: 0.999993	valid_1's auc: 0.645805
[1500]	training's auc: 0.999999	valid_1's auc: 0.647038
[1800]	training's auc: 1	valid_1's auc: 0.647389
Early stopping, best iteration is:
[1747]	training's auc: 1	valid_1's auc: 0.647497


KeyboardInterrupt: 

In [165]:
#early stopping 200
pred, oof, cv, cv_std, best_feat = LGB_KFOLD_BINA(5, day_tr.drop('game_id',axis=1), day_te.drop('game_id',axis=1), 
                                                  "auc", lr=0.003, num_leaves=700, max_depth=-1, min_data_in_leaf=20)

fold 0--------------------------------------------------
Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.993862	valid_1's auc: 0.646748
[600]	training's auc: 0.999061	valid_1's auc: 0.650118
[900]	training's auc: 0.9999	valid_1's auc: 0.651281
[1200]	training's auc: 0.999994	valid_1's auc: 0.652789
[1500]	training's auc: 1	valid_1's auc: 0.653053
[1800]	training's auc: 1	valid_1's auc: 0.653566
[2100]	training's auc: 1	valid_1's auc: 0.653678
[2400]	training's auc: 1	valid_1's auc: 0.654099
Early stopping, best iteration is:
[2417]	training's auc: 1	valid_1's auc: 0.654141


KeyboardInterrupt: 

# Time span 5 bins

In [195]:
def make_timespan2(df):

    df['time_span'] = 0
    df['time_span'] = np.where(df['time']<2, 0, 
                                  np.where(df['time']<4, 1, 
                                          np.where(df['time']<6, 2,
                                                  np.where(df['time']<8, 3,
                                                          np.where(df['time']<10, 4, 5)))))
    return df

In [197]:
events = tr['event'].unique().tolist()

def event_timespan(df):
    # make timespan
    df = make_timespan2(df)
    new_p0 = pd.DataFrame({"game_id":df['game_id'].unique()})
    new_p1 = pd.DataFrame({"game_id":df['game_id'].unique()})
    # for player 0
    for eve_name in tqdm(events, total=len(events)):
        p0 = df[df['player']==0]
        tmp = p0[df['event']==eve_name].groupby(['game_id','event'])['time_span'].value_counts(dropna=False, normalize=False).unstack()
        tmp.columns = ['p0_{}_tsp_'.format(eve_name)+str(i) for i in range(0,6)]
        tmp.reset_index(inplace=True)
        tmp.fillna(0, inplace=True)
        tmp.drop(['event'],axis=1,inplace=True)
        
        new_p0 = new_p0.merge(tmp, on=['game_id'],how='left')
        del tmp;gc.collect()
    del p0;gc.collect()
    
    # for player 0
    for eve_name in tqdm(events, total=len(events)):
        p1 = df[df['player']==1]
        tmp = p1[df['event']==eve_name].groupby(['game_id','event'])['time_span'].value_counts(dropna=False, normalize=False).unstack()
        tmp.columns = ['p1_{}_tsp_'.format(eve_name)+str(i) for i in range(0,6)]
        tmp.reset_index(inplace=True)
        tmp.fillna(0, inplace=True)
        tmp.drop(['event'],axis=1,inplace=True)
        
        new_p1 = new_p1.merge(tmp, on=['game_id'],how='left')
    del p1;gc.collect()
    
    new_df = new_p0.merge(new_p1, on=['game_id'], how='left')
    
    return new_df

In [198]:
%%time
tr_eve2 = event_timespan(tr)
te_eve2 = event_timespan(te)

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]c:\users\roal2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:30<00:00,  3.80s/it]


Wall time: 3min 29s


In [199]:
print(tr_eve2.shape, te_eve2.shape)

(38872, 97) (16787, 97)


In [200]:
tr_eve2.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Selection_tsp_0,p0_Selection_tsp_1,p0_Selection_tsp_2,...,p1_AddToControlGroup_tsp_2,p1_AddToControlGroup_tsp_3,p1_AddToControlGroup_tsp_4,p1_AddToControlGroup_tsp_5,p1_ControlGroup_tsp_0,p1_ControlGroup_tsp_1,p1_ControlGroup_tsp_2,p1_ControlGroup_tsp_3,p1_ControlGroup_tsp_4,p1_ControlGroup_tsp_5
0,0,82.0,146.0,121.0,95.0,0.0,0.0,11.0,10.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,59.0,131.0,135.0,136.0,166.0,0.0,78.0,23.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,71.0,64.0,89.0,70.0,119.0,0.0,19.0,20.0,19.0,...,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,3,70.0,134.0,173.0,153.0,183.0,0.0,32.0,23.0,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,99.0,184.0,147.0,0.0,0.0,0.0,12.0,26.0,29.0,...,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [201]:
tr_eve2.filter(regex='p0').columns

Index(['p0_Camera_tsp_0', 'p0_Camera_tsp_1', 'p0_Camera_tsp_2',
       'p0_Camera_tsp_3', 'p0_Camera_tsp_4', 'p0_Camera_tsp_5',
       'p0_Selection_tsp_0', 'p0_Selection_tsp_1', 'p0_Selection_tsp_2',
       'p0_Selection_tsp_3', 'p0_Selection_tsp_4', 'p0_Selection_tsp_5',
       'p0_Ability_tsp_0', 'p0_Ability_tsp_1', 'p0_Ability_tsp_2',
       'p0_Ability_tsp_3', 'p0_Ability_tsp_4', 'p0_Ability_tsp_5',
       'p0_Right Click_tsp_0', 'p0_Right Click_tsp_1', 'p0_Right Click_tsp_2',
       'p0_Right Click_tsp_3', 'p0_Right Click_tsp_4', 'p0_Right Click_tsp_5',
       'p0_SetControlGroup_tsp_0', 'p0_SetControlGroup_tsp_1',
       'p0_SetControlGroup_tsp_2', 'p0_SetControlGroup_tsp_3',
       'p0_SetControlGroup_tsp_4', 'p0_SetControlGroup_tsp_5',
       'p0_GetControlGroup_tsp_0', 'p0_GetControlGroup_tsp_1',
       'p0_GetControlGroup_tsp_2', 'p0_GetControlGroup_tsp_3',
       'p0_GetControlGroup_tsp_4', 'p0_GetControlGroup_tsp_5',
       'p0_AddToControlGroup_tsp_0', 'p0_AddToControlGro

In [203]:
tr_eve2.filter(regex='p1').columns

Index(['p1_Camera_tsp_0', 'p1_Camera_tsp_1', 'p1_Camera_tsp_2',
       'p1_Camera_tsp_3', 'p1_Camera_tsp_4', 'p1_Camera_tsp_5',
       'p1_Selection_tsp_0', 'p1_Selection_tsp_1', 'p1_Selection_tsp_2',
       'p1_Selection_tsp_3', 'p1_Selection_tsp_4', 'p1_Selection_tsp_5',
       'p1_Ability_tsp_0', 'p1_Ability_tsp_1', 'p1_Ability_tsp_2',
       'p1_Ability_tsp_3', 'p1_Ability_tsp_4', 'p1_Ability_tsp_5',
       'p1_Right Click_tsp_0', 'p1_Right Click_tsp_1', 'p1_Right Click_tsp_2',
       'p1_Right Click_tsp_3', 'p1_Right Click_tsp_4', 'p1_Right Click_tsp_5',
       'p1_SetControlGroup_tsp_0', 'p1_SetControlGroup_tsp_1',
       'p1_SetControlGroup_tsp_2', 'p1_SetControlGroup_tsp_3',
       'p1_SetControlGroup_tsp_4', 'p1_SetControlGroup_tsp_5',
       'p1_GetControlGroup_tsp_0', 'p1_GetControlGroup_tsp_1',
       'p1_GetControlGroup_tsp_2', 'p1_GetControlGroup_tsp_3',
       'p1_GetControlGroup_tsp_4', 'p1_GetControlGroup_tsp_5',
       'p1_AddToControlGroup_tsp_0', 'p1_AddToControlGro

In [204]:
p0_cols = tr_eve2[tr_eve2.filter(regex='p0').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()
p1_cols = tr_eve2[tr_eve2.filter(regex='p1').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()

print(len(p0_cols),len(p1_cols))

48 48


In [205]:
tr_eve2.fillna(0 , inplace=True)
te_eve2.fillna(0 , inplace=True)

In [206]:
tr_eve2.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Selection_tsp_0,p0_Selection_tsp_1,p0_Selection_tsp_2,...,p1_AddToControlGroup_tsp_2,p1_AddToControlGroup_tsp_3,p1_AddToControlGroup_tsp_4,p1_AddToControlGroup_tsp_5,p1_ControlGroup_tsp_0,p1_ControlGroup_tsp_1,p1_ControlGroup_tsp_2,p1_ControlGroup_tsp_3,p1_ControlGroup_tsp_4,p1_ControlGroup_tsp_5
0,0,82.0,146.0,121.0,95.0,0.0,0.0,11.0,10.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,59.0,131.0,135.0,136.0,166.0,0.0,78.0,23.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,71.0,64.0,89.0,70.0,119.0,0.0,19.0,20.0,19.0,...,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,3,70.0,134.0,173.0,153.0,183.0,0.0,32.0,23.0,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,99.0,184.0,147.0,0.0,0.0,0.0,12.0,26.0,29.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [207]:
def diff_all(df):
    p0_cols = df[df.filter(regex='p0').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()
    p1_cols = df[df.filter(regex='p1').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()
    
    for a, b in tqdm(zip(p0_cols, p1_cols), total=len(p0_cols)):
        df["{}_{}_diff".format(a,b)] = df[a] - df[b]
    
    return df

In [208]:
%%time
tr_eve2 = diff_all(tr_eve2)
te_eve2 = diff_all(te_eve2)

100%|████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 1672.65it/s]

Wall time: 130 ms


In [209]:
tr_eve2.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Selection_tsp_0,p0_Selection_tsp_1,p0_Selection_tsp_2,...,p0_AddToControlGroup_tsp_2_p1_AddToControlGroup_tsp_2_diff,p0_AddToControlGroup_tsp_3_p1_AddToControlGroup_tsp_3_diff,p0_AddToControlGroup_tsp_4_p1_AddToControlGroup_tsp_4_diff,p0_AddToControlGroup_tsp_5_p1_AddToControlGroup_tsp_5_diff,p0_ControlGroup_tsp_0_p1_ControlGroup_tsp_0_diff,p0_ControlGroup_tsp_1_p1_ControlGroup_tsp_1_diff,p0_ControlGroup_tsp_2_p1_ControlGroup_tsp_2_diff,p0_ControlGroup_tsp_3_p1_ControlGroup_tsp_3_diff,p0_ControlGroup_tsp_4_p1_ControlGroup_tsp_4_diff,p0_ControlGroup_tsp_5_p1_ControlGroup_tsp_5_diff
0,0,82.0,146.0,121.0,95.0,0.0,0.0,11.0,10.0,14.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,59.0,131.0,135.0,136.0,166.0,0.0,78.0,23.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,71.0,64.0,89.0,70.0,119.0,0.0,19.0,20.0,19.0,...,-1.0,3.0,-1.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0
3,3,70.0,134.0,173.0,153.0,183.0,0.0,32.0,23.0,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,99.0,184.0,147.0,0.0,0.0,0.0,12.0,26.0,29.0,...,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [210]:
new_tr2 = tr_eve2.merge(tr_sp, on='game_id', how='left')
new_te2 = te_eve2.merge(te_sp, on='game_id', how='left')

In [211]:
new_tr2.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Camera_tsp_3,p0_Camera_tsp_4,p0_Camera_tsp_5,p0_Selection_tsp_0,p0_Selection_tsp_1,p0_Selection_tsp_2,...,game_lat_lon_le,p0_spc,p1_spc,game_spc,p0_latlon_spc,p1_latlon_spc,game_latlon_spc,p0_latlon_spc_le,p1_latlon_spc_le,game_latlon_spc_le
0,0,82.0,146.0,121.0,95.0,0.0,0.0,11.0,10.0,14.0,...,613,T,T,TT,145.2521.51T,22.75147.01T,145.2521.51T22.75147.01T,359,859,883
1,1,59.0,131.0,135.0,136.0,166.0,0.0,78.0,23.0,25.0,...,476,P,T,PT,140.0162.01P,28.018.51T,140.0162.01P28.018.51T,284,1156,650
2,2,71.0,64.0,89.0,70.0,119.0,0.0,19.0,20.0,19.0,...,853,P,Z,PZ,151.2520.51P,24.75128.01Z,151.2520.51P24.75128.01Z,492,995,1148
3,3,70.0,134.0,173.0,153.0,183.0,0.0,32.0,23.0,54.0,...,215,T,P,TP,127.2527.51T,24.75137.01P,127.2527.51T24.75137.01P,125,1049,295
4,4,99.0,184.0,147.0,0.0,0.0,0.0,12.0,26.0,29.0,...,2375,T,Z,TZ,36.75122.01T,163.2518.51Z,36.75122.01T163.2518.51Z,1308,720,3275


In [212]:
new_tr2.select_dtypes(include=[object]).columns.tolist()

['player0_starting',
 'player1_starting',
 'p0_lat_lon',
 'p1_lat_lon',
 'game_lat_lon',
 'p0_spc',
 'p1_spc',
 'game_spc',
 'p0_latlon_spc',
 'p1_latlon_spc',
 'game_latlon_spc']

In [213]:
def category_transform(tr_df, te_df):
    tr, te = df_copy(tr_df, te_df)
    cate_col = tr.select_dtypes(include=[object]).columns.tolist()
    for df in [tr, te]:
        for col in cate_col:
            df[col] = df[col].astype('category')
    return tr, te

In [214]:
new_tr2, new_te2 = category_transform(new_tr2, new_te2)

In [216]:
new_tr2.dtypes.value_counts()

float64     148
int32         6
category      2
int64         1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
dtype: int64

In [217]:
#early stopping 200
pred, oof, cv, cv_std, best_feat = LGB_KFOLD_BINA(5, new_tr2.drop('game_id',axis=1), new_te2.drop('game_id',axis=1), 
                                                  "auc", lr=0.003, num_leaves=700, max_depth=-1, min_data_in_leaf=20)

fold 0--------------------------------------------------
Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.986883	valid_1's auc: 0.648131
[600]	training's auc: 0.997054	valid_1's auc: 0.651836
[900]	training's auc: 0.999503	valid_1's auc: 0.652929
[1200]	training's auc: 0.99994	valid_1's auc: 0.653635
[1500]	training's auc: 0.999994	valid_1's auc: 0.654149
[1800]	training's auc: 1	valid_1's auc: 0.655073
[2100]	training's auc: 1	valid_1's auc: 0.655354
Early stopping, best iteration is:
[2074]	training's auc: 1	valid_1's auc: 0.65538
fold 1--------------------------------------------------
Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.987716	valid_1's auc: 0.663597
Early stopping, best iteration is:
[352]	training's auc: 0.990162	valid_1's auc: 0.664362
fold 2--------------------------------------------------
Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.986557	valid_1's 

# Time span 3 bins

In [218]:
def make_timespan3(df):

    df['time_span'] = 0
    df['time_span'] = np.where(df['time']<4, 0, 
                                  np.where(df['time']<8, 1, 2))
    return df

In [222]:
events = tr['event'].unique().tolist()

def event_timespan(df):
    # make timespan
    df = make_timespan3(df)
    new_p0 = pd.DataFrame({"game_id":df['game_id'].unique()})
    new_p1 = pd.DataFrame({"game_id":df['game_id'].unique()})
    # for player 0
    for eve_name in tqdm(events, total=len(events)):
        p0 = df[df['player']==0]
        tmp = p0[df['event']==eve_name].groupby(['game_id','event'])['time_span'].value_counts(dropna=False, normalize=False).unstack()
        tmp.columns = ['p0_{}_tsp_'.format(eve_name)+str(i) for i in range(0,3)]
        tmp.reset_index(inplace=True)
        tmp.fillna(0, inplace=True)
        tmp.drop(['event'],axis=1,inplace=True)
        
        new_p0 = new_p0.merge(tmp, on=['game_id'],how='left')
        del tmp;gc.collect()
    del p0;gc.collect()
    
    # for player 0
    for eve_name in tqdm(events, total=len(events)):
        p1 = df[df['player']==1]
        tmp = p1[df['event']==eve_name].groupby(['game_id','event'])['time_span'].value_counts(dropna=False, normalize=False).unstack()
        tmp.columns = ['p1_{}_tsp_'.format(eve_name)+str(i) for i in range(0,3)]
        tmp.reset_index(inplace=True)
        tmp.fillna(0, inplace=True)
        tmp.drop(['event'],axis=1,inplace=True)
        
        new_p1 = new_p1.merge(tmp, on=['game_id'],how='left')
    del p1;gc.collect()
    
    new_df = new_p0.merge(new_p1, on=['game_id'], how='left')
    
    return new_df

In [223]:
%%time
tr_eve3 = event_timespan(tr)
te_eve3 = event_timespan(te)

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]c:\users\roal2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:29<00:00,  3.73s/it]


Wall time: 3min 25s


In [224]:
print(tr_eve3.shape, te_eve3.shape)

(38872, 49) (16787, 49)


In [225]:
tr_eve3.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Selection_tsp_0,p0_Selection_tsp_1,p0_Selection_tsp_2,p0_Ability_tsp_0,p0_Ability_tsp_1,p0_Ability_tsp_2,...,p1_SetControlGroup_tsp_2,p1_GetControlGroup_tsp_0,p1_GetControlGroup_tsp_1,p1_GetControlGroup_tsp_2,p1_AddToControlGroup_tsp_0,p1_AddToControlGroup_tsp_1,p1_AddToControlGroup_tsp_2,p1_ControlGroup_tsp_0,p1_ControlGroup_tsp_1,p1_ControlGroup_tsp_2
0,0,228.0,216.0,0.0,21.0,29.0,0.0,18.0,16.0,0.0,...,0.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,190.0,271.0,166.0,101.0,53.0,32.0,23.0,36.0,18.0,...,0.0,11.0,15.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,135.0,159.0,119.0,39.0,34.0,17.0,20.0,31.0,18.0,...,2.0,21.0,55.0,33.0,1.0,2.0,2.0,0.0,0.0,2.0
3,3,204.0,326.0,183.0,55.0,94.0,31.0,15.0,48.0,19.0,...,3.0,317.0,324.0,69.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,283.0,147.0,0.0,38.0,29.0,0.0,34.0,23.0,0.0,...,0.0,81.0,18.0,0.0,2.0,2.0,0.0,NaN,NaN,NaN


In [226]:
tr_eve3.filter(regex='p0').columns

Index(['p0_Camera_tsp_0', 'p0_Camera_tsp_1', 'p0_Camera_tsp_2',
       'p0_Selection_tsp_0', 'p0_Selection_tsp_1', 'p0_Selection_tsp_2',
       'p0_Ability_tsp_0', 'p0_Ability_tsp_1', 'p0_Ability_tsp_2',
       'p0_Right Click_tsp_0', 'p0_Right Click_tsp_1', 'p0_Right Click_tsp_2',
       'p0_SetControlGroup_tsp_0', 'p0_SetControlGroup_tsp_1',
       'p0_SetControlGroup_tsp_2', 'p0_GetControlGroup_tsp_0',
       'p0_GetControlGroup_tsp_1', 'p0_GetControlGroup_tsp_2',
       'p0_AddToControlGroup_tsp_0', 'p0_AddToControlGroup_tsp_1',
       'p0_AddToControlGroup_tsp_2', 'p0_ControlGroup_tsp_0',
       'p0_ControlGroup_tsp_1', 'p0_ControlGroup_tsp_2'],
      dtype='object')

In [227]:
tr_eve3.filter(regex='p1').columns

Index(['p1_Camera_tsp_0', 'p1_Camera_tsp_1', 'p1_Camera_tsp_2',
       'p1_Selection_tsp_0', 'p1_Selection_tsp_1', 'p1_Selection_tsp_2',
       'p1_Ability_tsp_0', 'p1_Ability_tsp_1', 'p1_Ability_tsp_2',
       'p1_Right Click_tsp_0', 'p1_Right Click_tsp_1', 'p1_Right Click_tsp_2',
       'p1_SetControlGroup_tsp_0', 'p1_SetControlGroup_tsp_1',
       'p1_SetControlGroup_tsp_2', 'p1_GetControlGroup_tsp_0',
       'p1_GetControlGroup_tsp_1', 'p1_GetControlGroup_tsp_2',
       'p1_AddToControlGroup_tsp_0', 'p1_AddToControlGroup_tsp_1',
       'p1_AddToControlGroup_tsp_2', 'p1_ControlGroup_tsp_0',
       'p1_ControlGroup_tsp_1', 'p1_ControlGroup_tsp_2'],
      dtype='object')

In [228]:
p0_cols = tr_eve2[tr_eve3.filter(regex='p0').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()
p1_cols = tr_eve2[tr_eve3.filter(regex='p1').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()

print(len(p0_cols),len(p1_cols))

24 24


In [229]:
tr_eve3.fillna(0 , inplace=True)
te_eve3.fillna(0 , inplace=True)

In [230]:
tr_eve3.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Selection_tsp_0,p0_Selection_tsp_1,p0_Selection_tsp_2,p0_Ability_tsp_0,p0_Ability_tsp_1,p0_Ability_tsp_2,...,p1_SetControlGroup_tsp_2,p1_GetControlGroup_tsp_0,p1_GetControlGroup_tsp_1,p1_GetControlGroup_tsp_2,p1_AddToControlGroup_tsp_0,p1_AddToControlGroup_tsp_1,p1_AddToControlGroup_tsp_2,p1_ControlGroup_tsp_0,p1_ControlGroup_tsp_1,p1_ControlGroup_tsp_2
0,0,228.0,216.0,0.0,21.0,29.0,0.0,18.0,16.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,190.0,271.0,166.0,101.0,53.0,32.0,23.0,36.0,18.0,...,0.0,11.0,15.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,135.0,159.0,119.0,39.0,34.0,17.0,20.0,31.0,18.0,...,2.0,21.0,55.0,33.0,1.0,2.0,2.0,0.0,0.0,2.0
3,3,204.0,326.0,183.0,55.0,94.0,31.0,15.0,48.0,19.0,...,3.0,317.0,324.0,69.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,283.0,147.0,0.0,38.0,29.0,0.0,34.0,23.0,0.0,...,0.0,81.0,18.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0


In [231]:
def diff_all(df):
    p0_cols = df[df.filter(regex='p0').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()
    p1_cols = df[df.filter(regex='p1').columns.tolist()].select_dtypes(exclude=[object,'category']).columns.tolist()
    
    for a, b in tqdm(zip(p0_cols, p1_cols), total=len(p0_cols)):
        df["{}_{}_diff".format(a,b)] = df[a] - df[b]
    
    return df

In [232]:
%%time
tr_eve3 = diff_all(tr_eve3)
te_eve3 = diff_all(te_eve3)

100%|████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 1189.73it/s]

Wall time: 70.1 ms


In [233]:
tr_eve3.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Selection_tsp_0,p0_Selection_tsp_1,p0_Selection_tsp_2,p0_Ability_tsp_0,p0_Ability_tsp_1,p0_Ability_tsp_2,...,p0_SetControlGroup_tsp_2_p1_SetControlGroup_tsp_2_diff,p0_GetControlGroup_tsp_0_p1_GetControlGroup_tsp_0_diff,p0_GetControlGroup_tsp_1_p1_GetControlGroup_tsp_1_diff,p0_GetControlGroup_tsp_2_p1_GetControlGroup_tsp_2_diff,p0_AddToControlGroup_tsp_0_p1_AddToControlGroup_tsp_0_diff,p0_AddToControlGroup_tsp_1_p1_AddToControlGroup_tsp_1_diff,p0_AddToControlGroup_tsp_2_p1_AddToControlGroup_tsp_2_diff,p0_ControlGroup_tsp_0_p1_ControlGroup_tsp_0_diff,p0_ControlGroup_tsp_1_p1_ControlGroup_tsp_1_diff,p0_ControlGroup_tsp_2_p1_ControlGroup_tsp_2_diff
0,0,228.0,216.0,0.0,21.0,29.0,0.0,18.0,16.0,0.0,...,0.0,10.0,11.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1,190.0,271.0,166.0,101.0,53.0,32.0,23.0,36.0,18.0,...,0.0,121.0,5.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,135.0,159.0,119.0,39.0,34.0,17.0,20.0,31.0,18.0,...,-1.0,45.0,-34.0,-21.0,0.0,2.0,-1.0,0.0,0.0,-2.0
3,3,204.0,326.0,183.0,55.0,94.0,31.0,15.0,48.0,19.0,...,-3.0,-213.0,-308.0,-57.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,283.0,147.0,0.0,38.0,29.0,0.0,34.0,23.0,0.0,...,0.0,61.0,64.0,0.0,-1.0,-2.0,0.0,0.0,0.0,0.0


In [234]:
new_tr3 = tr_eve3.merge(tr_sp, on='game_id', how='left')
new_te3 = te_eve3.merge(te_sp, on='game_id', how='left')

In [235]:
new_tr3.head()

,game_id,p0_Camera_tsp_0,p0_Camera_tsp_1,p0_Camera_tsp_2,p0_Selection_tsp_0,p0_Selection_tsp_1,p0_Selection_tsp_2,p0_Ability_tsp_0,p0_Ability_tsp_1,p0_Ability_tsp_2,...,game_lat_lon_le,p0_spc,p1_spc,game_spc,p0_latlon_spc,p1_latlon_spc,game_latlon_spc,p0_latlon_spc_le,p1_latlon_spc_le,game_latlon_spc_le
0,0,228.0,216.0,0.0,21.0,29.0,0.0,18.0,16.0,0.0,...,613,T,T,TT,145.2521.51T,22.75147.01T,145.2521.51T22.75147.01T,359,859,883
1,1,190.0,271.0,166.0,101.0,53.0,32.0,23.0,36.0,18.0,...,476,P,T,PT,140.0162.01P,28.018.51T,140.0162.01P28.018.51T,284,1156,650
2,2,135.0,159.0,119.0,39.0,34.0,17.0,20.0,31.0,18.0,...,853,P,Z,PZ,151.2520.51P,24.75128.01Z,151.2520.51P24.75128.01Z,492,995,1148
3,3,204.0,326.0,183.0,55.0,94.0,31.0,15.0,48.0,19.0,...,215,T,P,TP,127.2527.51T,24.75137.01P,127.2527.51T24.75137.01P,125,1049,295
4,4,283.0,147.0,0.0,38.0,29.0,0.0,34.0,23.0,0.0,...,2375,T,Z,TZ,36.75122.01T,163.2518.51Z,36.75122.01T163.2518.51Z,1308,720,3275


In [236]:
new_tr3.select_dtypes(include=[object]).columns.tolist()

['player0_starting',
 'player1_starting',
 'p0_lat_lon',
 'p1_lat_lon',
 'game_lat_lon',
 'p0_spc',
 'p1_spc',
 'game_spc',
 'p0_latlon_spc',
 'p1_latlon_spc',
 'game_latlon_spc']

In [237]:
def category_transform(tr_df, te_df):
    tr, te = df_copy(tr_df, te_df)
    cate_col = tr.select_dtypes(include=[object]).columns.tolist()
    for df in [tr, te]:
        for col in cate_col:
            df[col] = df[col].astype('category')
    return tr, te

In [238]:
new_tr3, new_te3 = category_transform(new_tr3, new_te3)

In [239]:
new_tr3.dtypes.value_counts()

float64     76
int32        6
category     2
int64        1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
category     1
dtype: int64

In [240]:
#early stopping 200
pred, oof, cv, cv_std, best_feat = LGB_KFOLD_BINA(5, new_tr3.drop('game_id',axis=1), new_te3.drop('game_id',axis=1), 
                                                  "auc", lr=0.003, num_leaves=700, max_depth=-1, min_data_in_leaf=20)

fold 0--------------------------------------------------
Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.979637	valid_1's auc: 0.647801
[600]	training's auc: 0.993496	valid_1's auc: 0.649996
[900]	training's auc: 0.998435	valid_1's auc: 0.651174
Early stopping, best iteration is:
[897]	training's auc: 0.998416	valid_1's auc: 0.651199
fold 1--------------------------------------------------
Training until validation scores don't improve for 200 rounds
[300]	training's auc: 0.981086	valid_1's auc: 0.660909
[600]	training's auc: 0.994063	valid_1's auc: 0.661805


KeyboardInterrupt: 